# Analysis of spectral content of letter distortions

The following notebook quantifies the changes in the spatial and orientation energy caused by applying (1) Bandpass Noise and (2) Radial Frequency distortions to Sloan letters. It accompanies the paper 

Wallis, Tobias, Bethge & Wichmann (under review). Detecting distortions of peripherally-presented letter stimuli under crowded conditions.

This notebook relies on version 1.3.1 of Tom's `psyutils` package (unlike Saskia's code, which uses v.0.1.1 [available from Github]).


In [ ]:
# Import all the necessary packages
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import color, io, img_as_float, transform
import pandas as pd
import psyutils as pu
from psyutils.image import show_im
from itertools import product

%matplotlib inline

# # set some styles we like:
# sns.set_style("white")
# sns.set_style("ticks")
# sns.set_context("notebook")

In [ ]:
# set random seed:
rng = np.random.RandomState(seed = 22239217)

In [ ]:
top_dir = "/Users/tomwallis/Dropbox/Projects/letter-distortion-detection"
out_dir = os.path.join(top_dir, 'results', 'spectral_analysis')


## Helper functions for making letter images and distorting them

Here I duplicate the stimulus generation code for distortion from Saskia's scripts (see `experiment1.py`), updated to the latest version of psyutils (e.g. replaced `make_filter` with `make_filter_log_exp`, changed norming of filter to match old code).

In [ ]:
def bex_distorted_im(im, amplitude=2, frequency=4):
    """A spatial distortion method based on a method by Peter Bex (see ref, below).

    Args:
        im (float): the image to distort.
        amplitude (int): determines the amplitude of distortion in pixels.
        frequency (int): peak frequency of the filter.
    Returns:
        dist_image (float): the distorted image.

    Example:
        Distort an image:
            im = img_as_float(pu.im_data.tiger_grey())
            scale = 5
            f_peak = 4
            dist_im = bex_distorted_im(im, scale, f_peak) 
            pu.image.show_im(dist_im)

    Reference:
        Bex, P. J. (2010). (In) sensitivity to spatial distortion in natural
        scenes. Journal of Vision, 10(2), 23:1-15.

    """
    # log-exponential filter to create random-bandpass filtered noise samples as positional offset
    filt = pu.image.make_filter_log_exp(size=im.shape[0], 
                                        peak=frequency, 
                                        width=0.5)
    
    # cosine window that reduces to zero over the padding region 
    cos_win = pu.image.cos_win_2d(size=im.shape[0], ramp=14, 
                                  ramp_type = "pixels")


    # old version of psyutils (used in sakia's code) scales filtered noise
    # to have max absolute value of 1. Do this manually here:
    filtered_noise_x = pu.image.make_filtered_noise(filt, rng)
    filtered_noise_y = pu.image.make_filtered_noise(filt, rng)
    
    filtered_noise_x = filtered_noise_x / abs(filtered_noise_x).max()
    filtered_noise_y = filtered_noise_y / abs(filtered_noise_y).max()

    # horizontal and vertical positional offset 
    filt_noise_x = filtered_noise_x * cos_win * amplitude
    filt_noise_y = filtered_noise_y * cos_win * amplitude
    
    # disort image 
    dist_im = pu.image.grid_distort(im, x_offset=filt_noise_x, y_offset=filt_noise_y, 
                                   method="linear", fill_method=1)
    return(dist_im)


def rf_distorted_im(im, amplitude=0.1, frequency=3):
    """Creates a radial frequency modulated grid by modulating the distance from the center to every point 
    sinusoidally with a certain amplitude and frequency
    
    Based on a method by Dickinson et al. (see ref, below). 

    Args:
        im (float): the image to distort.
        amplitude (float): modulation amplitude, expressed as a proportion of the distance from the center of the 
                   unmodulated radius
        frequency (int): the frequency of modulation in 2*pi radians
        
    Returns:
        dist_image (float): the distorted image.

    Example:
        Distort an image:
            im = img_as_float(pu.im_data.tiger_grey())
            amplitude = 0.2
            frequency = 5
            dist_im = rf_distorted_im(im, amplitude, frequency) 
            pu.image.show_im(dist_im)

    Reference:
        Dickinson, J. E., Almeida, R. A., Bell, J. & Badcock, D. R. (2010). Global shape aftereffects have a local substrate: 
        A tilt aftereffect field. Journal of Vision,10 (13), 2.
    """

    # get radial distance 
    x = np.linspace(-20, 20, num=im.shape[0])
    xx, yy = np.meshgrid(x, x)    
    rad_dist = (xx**2 + yy**2)**0.5
    
    # randomise phase 
    rand_num = rng.rand()*2*np.pi

    # angular distance
    ang_dist = ((rand_num+np.arctan2(xx, -yy))%(2*np.pi))-np.pi
 
    # modulate distance of each point from the center sinusoidally (cf. Dickinson et al. p. 3)
    rf_grid = rad_dist*(1+amplitude*(np.sin(frequency*ang_dist)))
    
    # calculate radial distance offset for each point
    delta_rad = rf_grid - rad_dist
   
    # cosine window that reduces to zero over the padding region 
    cos_win = pu.image.cos_win_2d(size=im.shape[0], ramp=14,
                                  ramp_type="pixels")
    
    # convert from polar to cartesian coordinates 
    x_offset = delta_rad * np.cos(ang_dist) * cos_win
    y_offset = delta_rad * np.sin(ang_dist) * cos_win

    # distort image
    dist_im = pu.image.grid_distort(im, x_offset=x_offset, y_offset=y_offset, 
                                   method="linear", fill_method=1)
    return(dist_im)


def undistorted_letter(letter):
    """
    Return the undistorted sloan letter with appropriate padding as used in the experiment.
    
    """
    letter_dict = pu.im_data.sloan_letters()
    im = letter_dict[letter]

    # resize letter to have a padding area of 14 pixels at each side
    im = transform.resize(im, (64, 64))
    pad = np.ones((92,92))
    im = pu.image.put_rect_in_rect(im, pad)
    return im

## Example spectral analysis using psyutils functions

In [ ]:
img = undistorted_letter("K")
plt.figure(figsize=(15,5))
plt.subplot(1,3,1), plt.imshow(img, cmap='gray'), plt.axis('off')

A, P, F = pu.image.spectral_analysis_fft(img)

plt.subplot(1,3,2), plt.imshow(np.log(A), cmap='gray'), plt.axis('off')
plt.subplot(1,3,3), plt.imshow(P, cmap='gray'), plt.axis('off')

plt.show()

In [ ]:
res = pu.image.spectral_analysis(img)
pu.image.spectral_analysis_plot(res)

In [ ]:
# example bpn distortion:
d_im = bex_distorted_im(img, amplitude = 5, frequency=5)

plt.figure(figsize=(15,5))
plt.subplot(1,3,1), plt.imshow(d_im, cmap='gray'), plt.axis('off')

A, P, F = pu.image.spectral_analysis_fft(d_im)

plt.subplot(1,3,2), plt.imshow(np.log(A), cmap='gray'), plt.axis('off')
plt.subplot(1,3,3), plt.imshow(P, cmap='gray'), plt.axis('off')

plt.show()

In [ ]:
res = pu.image.spectral_analysis(d_im)
pu.image.spectral_analysis_plot(res)

In [ ]:
# example rf distortion:
d_im = rf_distorted_im(img, amplitude = 0.32, frequency=2)

plt.figure(figsize=(15,5))
plt.subplot(1,3,1), plt.imshow(d_im, cmap='gray'), plt.axis('off')

A, P, F = pu.image.spectral_analysis_fft(d_im)

plt.subplot(1,3,2), plt.imshow(np.log(A), cmap='gray'), plt.axis('off')
plt.subplot(1,3,3), plt.imshow(P, cmap='gray'), plt.axis('off')

plt.show()

In [ ]:
res = pu.image.spectral_analysis(d_im)
pu.image.spectral_analysis_plot(res)

# Which frequencies and amplitudes to test?

We wish to examine whether

1. different patterns of sensitivity for BPN and RF distortions could be explained by similar falloffs in spatial frequency or orientation energy
2. differences in letter performance could be explained similarly

To do this, I will 

1. take the mean threshold values from Experiment 1 in each of flanked / unflanked, BPN / RF, each frequency of distortion
2. compute the spectra for each undistorted letter, and for each letter with each threshold level of distortion. Distortions will be repeated some number of times to get an idea of the average effect of distortions.
3. Plot the spectra differences for each condition (passed to R for plotting).

In [ ]:
fname = os.path.join(top_dir, "results", "r-analysis-final-paper", "expt_1_thresholds.csv")
thresh_dat = pd.read_csv(fname)
thresh_dat.info()

In [ ]:
mean_threshs = thresh_dat.groupby(["flanked", "freq", "distortion"]).threshold.mean()
mean_threshs = pd.DataFrame(mean_threshs)
mean_threshs.reset_index(inplace=True)
print(mean_threshs)

**Note**: In the BPN distortion type, thresholds are in units of degrees (representing the amplitude of the shift in pixels -- see `/code/analysis/getpsignifitdata.m` line 107. Below I correct *back* to pixels to ensure that the shift is correct for applying the distortions.

In [ ]:
# add a third threshold level to the data above, representing the largest distortion 
# we applied:
extra_level_1 = pu.psydata.expand_grid({"flanked": ["max"],
                                        "freq": np.unique(mean_threshs[mean_threshs["distortion"]=="BPN"].freq),
                                        "distortion": ["BPN"]})
extra_level_1["threshold"] = 5 / 41.5  # largest threshold used for BPN

extra_level_2 = pu.psydata.expand_grid({"flanked": ["max"],
                                        "freq": np.unique(mean_threshs[mean_threshs["distortion"]=="RF"].freq),
                                        "distortion": ["RF"]})
extra_level_2["threshold"] = 0.32

mean_threshs = mean_threshs.append(extra_level_1, ignore_index = True)
mean_threshs = mean_threshs.append(extra_level_2, ignore_index = True)
mean_threshs

In [ ]:
# mean thresholds over frequencies:
mean_threshs.groupby(["flanked", "distortion"]).threshold.mean()

## Loop over conditions

Saving results to a big dataframe.

In [ ]:
def append_energy_results(sf_df, ori_df, res,
                         letter_type, 
                         letter,
                         flanked,
                         freq, 
                         distortion,
                         rep):
    """ function to append the energy result res
    to dataframe df.
    
    """
    
    # spatial freq
    x = res['freq']
    y = res['freq_amp']

    this_df = pd.DataFrame({'letter_type': letter_type,
                            'letter': letter,
                            'flanked': flanked,
                            'freq': freq,
                            'distortion': distortion,
                            'rep': rep,
                            'x': x,
                            'y': y})
    sf_df = sf_df.append(this_df, ignore_index=True)
    
    # ori
    x = res['ang']
    y = res['ang_amp']

    this_df = pd.DataFrame({'letter_type': letter_type,
                            'letter': letter,
                            'flanked': flanked,
                            'freq': freq,
                            'distortion': distortion,
                            'rep': rep,
                            'x': x,
                            'y': y})
    ori_df = ori_df.append(this_df, ignore_index=True)    
    
    return sf_df, ori_df
    

In [ ]:
letters = ["K", "H", "D", "N"]
# letters = ["K"]
reps = 15  # number of repetitions of distortions (to get average distortion effect).
sf_energy = pd.DataFrame()
ori_energy = pd.DataFrame()

for letter in letters:
    print(letter)
    # do distortions:
    for idx in range(len(mean_threshs)):
        flank = mean_threshs.ix[idx, "flanked"]
        freq = mean_threshs.ix[idx, "freq"]
        dist = mean_threshs.ix[idx, "distortion"]
        thresh = mean_threshs.ix[idx, "threshold"]
        px_thresh = thresh * 41.5  # see comment above: thresholds --> pixel units        

        im = undistorted_letter(letter)
        res = pu.image.spectral_analysis(im)

        # duplicate original spectrum in each condition to make plotting easier:
        sf_energy, ori_energy = append_energy_results(sf_energy, ori_energy,
                                                      res,
                                                      letter_type="Undistorted",
                                                      letter=letter,
                                                      flanked=flank,
                                                      freq=freq,
                                                      distortion=dist,
                                                      rep=np.nan)        
        
        for rep in range(reps):
            if dist == "BPN":
                d_im = bex_distorted_im(im, amplitude=px_thresh, frequency=freq)
            elif dist == "RF":
                d_im = rf_distorted_im(im, amplitude=thresh, frequency=freq)
            else:
                raise ValueError("distortion not known")

#             # save images to check:
#             if flank is "max":
#                 fname = os.path.join(out_dir, "test_ims", 
#                                     "{}_{}_{}_{}_{}_{}.png".format(
#                     dist, letter, freq, thresh, flank, rep))
#                 io.imsave(fname, d_im)
                
            res = pu.image.spectral_analysis(d_im)
            sf_energy, ori_energy = append_energy_results(sf_energy, ori_energy,
                                                          res,
                                                          letter_type="Distorted",
                                                          letter=letter,
                                                          flanked=flank,
                                                          freq=freq,
                                                          distortion=dist,
                                                          rep=rep)        

In [ ]:
#save data:
sf_energy.to_csv(os.path.join(out_dir, "sf_energy.csv"))
ori_energy.to_csv(os.path.join(out_dir, "ori_energy.csv"))